In [4]:
import sys
import keras
from keras import backend as K
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
# from keras.preprocessing import image
import keras.utils as image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import tensorflow as tf

import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
import matplotlib.pyplot as plt

from tensorflow.keras.layers.experimental import preprocessing

In [5]:
# DIMENSIONS AND CONSTANT

SIZE_X = 224
SIZE_Y = 224

BATCH_SIZE = 4
SEED=1

In [6]:
mobile = keras.applications.mobilenet.MobileNet()  

In [7]:
base_model=MobileNet(weights='imagenet',include_top=False, input_shape=(224, 224, 3)) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(3,activation='softmax')(x) #final layer with softmax activation


In [8]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [9]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [10]:
#for i,layer in enumerate(model.layers):
#  print(i,layer.name, layer.trainable)

In [8]:
#train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

#train_generator=train_datagen.flow_from_directory('/home/agtonybarletta/projects/posture_check/data',
#                                                 target_size=(224,224),
#                                                 color_mode='rgb',
#                                                 batch_size=BATCH_SIZE,
#                                                 class_mode='categorical',
#                                                 shuffle=True)


In [10]:
validation_ds = keras.utils.image_dataset_from_directory(
    directory='/home/agtonybarletta/projects/posture_check/data',
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(224, 224),
    color_mode="rgb",
    validation_split=0.2,
    subset="validation",
    seed=1,
    shuffle = True
)
validation_ds
training_ds = keras.utils.image_dataset_from_directory(
    directory='/home/agtonybarletta/projects/posture_check/data',
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(224, 224),
    color_mode="rgb",
    validation_split=0.2,
    subset="training",
    seed=1,
    shuffle = True
)
training_ds

Found 174 files belonging to 3 classes.
Using 34 files for validation.
Found 174 files belonging to 3 classes.
Using 140 files for training.


<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None))>

In [11]:
def print_ds_info(dataset):
    # Create a tf.data.Dataset object
    
    # Retrieve shape information using element_spec
    input_shape = dataset.element_spec[0].shape
    output_shape = dataset.element_spec[1].shape
    
    # Print the shape information
    print("Input shape:", input_shape)
    print("Output shape:", output_shape)
    
    # Get the number of samples
    num_samples = len(dataset)
    print("Number of samples batched:", num_samples)

    # Get the number of samples
    num_samples = len(dataset)*BATCH_SIZE
    print("Number of samples total:", num_samples)
print_ds_info(training_ds)

NameError: name 'training_ds' is not defined

In [12]:
"""
# show example data


# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    training_ds
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

plt.figure(figsize=(10,10))
training_ds_generator_f = iter(training_ds.unbatch().map(lambda x, y: x/224).batch(1))
ex = next(training_ds_generator_f)
for i in range(16):
    image = ex
    plt.subplot(4, 4, i+1)
    plt.imshow(tf.squeeze(image))
    plt.axis('off')
plt.show()
"""

"\n# show example data\n\n\n# Data Pipeline\ndef convert_to_float(image, label):\n    image = tf.image.convert_image_dtype(image, dtype=tf.float32)\n    return image, label\n\nAUTOTUNE = tf.data.experimental.AUTOTUNE\nds_train = (\n    training_ds\n    .map(convert_to_float)\n    .cache()\n    .prefetch(buffer_size=AUTOTUNE)\n)\n\nplt.figure(figsize=(10,10))\ntraining_ds_generator_f = iter(training_ds.unbatch().map(lambda x, y: x/224).batch(1))\nex = next(training_ds_generator_f)\nfor i in range(16):\n    image = ex\n    plt.subplot(4, 4, i+1)\n    plt.imshow(tf.squeeze(image))\n    plt.axis('off')\nplt.show()\n"

In [12]:
def show_sample(ds):
    samples = ds.unbatch().map(lambda x, y: x/224).batch(1)
    gen = iter(samples)
    for i in range(min(len(list(samples)), 16)):
        image = next(gen)
        plt.subplot(4, 4, i+1)
        plt.imshow(tf.squeeze(image))
        plt.axis('off')
    plt.show()

In [14]:
#augment
augment = keras.Sequential([
    preprocessing.RandomContrast(factor=0.5),
    preprocessing.RandomFlip(mode='horizontal'), # meaning, left-to-right
    # preprocessing.RandomFlip(mode='vertical'), # meaning, top-to-bottom
    # preprocessing.RandomWidth(factor=0.15), # horizontal stretch
    preprocessing.RandomRotation(factor=0.05),
    preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
])
AUTOTUNE = tf.data.AUTOTUNE

augmented_training_ds= training_ds.map(lambda x, y: (augment(x), y))

new_training_ds = training_ds \
.concatenate(augmented_training_ds) \
.concatenate(augmented_training_ds) \
.concatenate(augmented_training_ds)
print_ds_info(training_ds)
print_ds_info(augmented_training_ds)
print_ds_info(new_training_ds)

Input shape: (None, 224, 224, 3)
Output shape: (None, 3)
Number of samples batched: 35
Number of samples total: 140
Input shape: (None, 224, 224, 3)
Output shape: (None, 3)
Number of samples batched: 35
Number of samples total: 140
Input shape: (None, 224, 224, 3)
Output shape: (None, 3)
Number of samples batched: 140
Number of samples total: 560


In [15]:
"""
small_ds = training_ds.take(1)
print_ds_info(training_ds)
small_aug_ds = small_ds.map(lambda x, y: (augment(x), y))
print_ds_info(small_aug_ds)

new_small_ds = small_ds.concatenate(small_aug_ds).concatenate(small_aug_ds).concatenate(small_aug_ds)

show_sample(small_ds)
show_sample(small_aug_ds)
show_sample(new_small_ds)
"""

'\nsmall_ds = training_ds.take(1)\nprint_ds_info(training_ds)\nsmall_aug_ds = small_ds.map(lambda x, y: (augment(x), y))\nprint_ds_info(small_aug_ds)\n\nnew_small_ds = small_ds.concatenate(small_aug_ds).concatenate(small_aug_ds).concatenate(small_aug_ds)\n\nshow_sample(small_ds)\nshow_sample(small_aug_ds)\nshow_sample(new_small_ds)\n'

In [16]:
# Initialize a dictionary to store the count for each class
class_names = training_ds.class_names
print(class_names)
class_count = {class_name: 0 for class_name in class_names}

# Iterate over the dataset and count the samples for each class
for _, label in training_ds:
    label_index = np.argmax(label.numpy())  # Convert one-hot encoded label to class index
    class_name = class_names[label_index]  # Get the class name corresponding to the class index
    class_count[class_name] += 1

# Print the count for each class
for class_name, count in class_count.items():
    print(f"Class '{class_name}' has {count} samples.")

['correct', 'incorrect', 'none']


2023-07-08 19:37:49.980977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [140]
	 [[{{node Placeholder/_0}}]]
2023-07-08 19:37:49.981737: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [140]
	 [[{{node Placeholder/_0}}]]


Class 'correct' has 12 samples.
Class 'incorrect' has 18 samples.
Class 'none' has 5 samples.


In [26]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=len(new_training_ds)
step_size_train
#model.fit_generator(generator=train_generator,
#                   steps_per_epoch=step_size_train,
#                   epochs=25)
class_weight = {0: 2,
                1: 1,
                2: 3}
history = model.fit(new_training_ds, validation_data=validation_ds, steps_per_epoch=step_size_train,
                  epochs=50, class_weight=class_weigth)
model.save_weights('my_checkpoint-shuffle2')


Epoch 1/50
  1/140 [..............................] - ETA: 11:19 - loss: 2.2430e-04 - accuracy: 1.0000

KeyboardInterrupt: 

In [13]:
model.load_weights('my_checkpoint-shuffle')

In [14]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [15]:


def print_class_probabilities(y_pred, class_labels):
    n_samples, n_classes = y_pred.shape
    
    for i in range(n_samples):
        probabilities = y_pred[i]
        for class_idx, probability in enumerate(probabilities):
            class_label = class_labels[class_idx]
            print(f' {class_label}: {"{:.3f}".format(probability)}', end='')
        print()

class_labels = training_ds.class_names

print_class_probabilities(pred, class_labels)

NameError: name 'training_ds' is not defined

In [1]:
#correct
#img_path = '/home/agtonybarletta/projects/posture_check/data/correct/opencv_frame_2023-07-07T13:05:22.608620.png'
#img_path = '/home/agtonybarletta/projects/posture-check-model/data/correct/opencv_frame_2023-07-07T13:05:22.608620.png'
#img_path = '/home/agtonybarletta/projects/posture_check/data/correct/opencv_frame_2023-03-06T12:26:08.022630.png'
#incorrect
img_path = '/home/agtonybarletta/projects/posture-check-model/data/correct/opencv_frame_2023-03-06T12:26:08.022630.png'
#img_path = '/home/agtonybarletta/projects/posture_check/data/incorrect/opencv_frame_2023-07-05T15:12:43.895980.png'
#img_path = '/home/agtonybarletta/projects/posture_check/data/incorrect/opencv_frame_2023-03-03T14:32:16.381431.png'
#Image(filename=img_path)

In [21]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(224, 224))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    #img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()
    return img_tensor

new_image = load_image(img_path)

pred = model.predict(new_image)
pred

1/1 [==============================] - 0s 56ms/step


array([[2.6258707e-02, 9.7373331e-01, 8.0610871e-06]], dtype=float32)

In [49]:
import cv2
import time
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

model.load_weights('my_checkpoint-shuffle')

def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    #img_array = image.img_to_array(img)
    #img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return img
  

img_counter = 0
start_time = time.time()
while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
    if time.time() - start_time >= 3:
        start_time = time.time()
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        # print("{} written!".format(img_name))
        preprocessed_image = load_image(img_name)
        predictions = model.predict(preprocessed_image)
        
        #results = imagenet_utils.decode_predictions(predictions)
        print_class_probabilities(predictions, class_labels)
cam.release()

cv2.destroyAllWindows()

1/1 [==============================] - 0s 29ms/step
 correct: 0.000 incorrect: 1.000 none: 0.000
1/1 [==============================] - 0s 27ms/step
 correct: 1.000 incorrect: 0.000 none: 0.000
1/1 [==============================] - 0s 27ms/step
 correct: 0.998 incorrect: 0.002 none: 0.000
1/1 [==============================] - 0s 27ms/step
 correct: 0.103 incorrect: 0.897 none: 0.000
1/1 [==============================] - 0s 26ms/step
 correct: 0.226 incorrect: 0.774 none: 0.000
1/1 [==============================] - 0s 26ms/step
 correct: 0.000 incorrect: 0.001 none: 0.999
1/1 [==============================] - 0s 26ms/step
 correct: 0.000 incorrect: 0.000 none: 1.000
1/1 [==============================] - 0s 28ms/step
 correct: 0.993 incorrect: 0.007 none: 0.000
1/1 [==============================] - 0s 26ms/step
 correct: 1.000 incorrect: 0.000 none: 0.000
1/1 [==============================] - 0s 31ms/step
 correct: 0.008 incorrect: 0.992 none: 0.000
1/1 [=========================

In [22]:
loss, acc = model.evaluate(validation_ds, verbose=2)

2023-07-08 19:34:37.180367: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [34]
	 [[{{node Placeholder/_4}}]]
2023-07-08 19:34:37.180860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [34]
	 [[{{node Placeholder/_4}}]]


9/9 - 1s - loss: 4.8719 - accuracy: 0.0000e+00 - 915ms/epoch - 102ms/step


In [ ]:
model.fit(training_ds, steps_per_epoch=step_size_train,
                  epochs=125)